# Lab 06 - BERT Finetuning

In [2]:
%pip install datasets
%pip install transformers
%pip install spacy

  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
   ---------------------------------------- 0.0/510.5 kB ? eta -:--:--
   --------------------------------------- 510.5/510.5 kB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/170.9 kB ? eta -:--:--
   --------------------------------------- 170.9/170.9 kB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/365.2 kB ? eta -:--:--
   --------------------------------------- 365.2/365.2 kB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/346.4 kB 

In [4]:
%pip install torch
%pip install spacy-transformers

  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/198.6 MB 9.6 MB/s eta 0:00:21
   ---------------------------------------- 1.6/198.6 MB 19.8 MB/s eta 0:00:10
    --------------------------------------- 2.9/198.6 MB 23.5 MB/s eta 0:00:09
    --------------------------------------- 4.0/198.6 MB 23.2 MB/s eta 0:00:09
   - -------------------------------------- 5.8/198.6 MB 26.4 MB/s eta 0:00:08
   - -------------------------------------- 6.7/198.6 MB 26.8 MB/s eta 0:00:08
   - -------------------------------------- 8.1/198.6 MB 27.1 MB/s eta 0:00:08
   - -------------------------------------- 9.8/198.6 MB 27.3 MB/s eta 0:00:07
   -- ------------------------------------- 11.2/198.6 MB 29.7 MB/s eta 0:00:07
   -- ------------------------------------- 12.7/198.6 MB 31.2 MB/s eta 0:00:06
   -- ------------------------------------- 14.0/198.6 MB 29.8 MB/s eta 0

In [7]:
from datasets import load_dataset, load_metric

dataset = load_dataset("surrey-nlp/PLOD-CW")

In [8]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
for i in dataset["train"]:
    print(i['tokens'])
    print(i['ner_tags'])
    break

['For', 'this', 'purpose', 'the', 'Gothenburg', 'Young', 'Persons', 'Empowerment', 'Scale', '(', 'GYPES', ')', 'was', 'developed', '.']
['B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O']


In [10]:
# !python -m spacy download en_core_web_trf

In [29]:
# import spacy

# nlp = spacy.load("en_core_web_trf")

In [11]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertTokenizer, BertModel, BertTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", )
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertModel.from_pretrained("bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=4)

c:\Users\gk00554\.conda\envs\nlp24\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
short_dataset = dataset["train"][:100]
# short_dataset["ner_tags"]

In [44]:
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [13]:
tokenized_input = tokenizer(short_dataset["tokens"], is_split_into_words=True)
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
for token in tokenized_input["input_ids"]:
    print(tokenizer.convert_ids_to_tokens(token))
    break

['[CLS]', 'for', 'this', 'purpose', 'the', 'gothenburg', 'young', 'persons', 'empowerment', 'scale', '(', 'g', '##ype', '##s', ')', 'was', 'developed', '.', '[SEP]']


In [14]:
# tokenized_input

In [15]:
label_encoding = {"B-O": 0, "B-AC": 1, "B-LF": 2, "I-LF": 3}

In [16]:
label_list = []

for sample in short_dataset["ner_tags"]:
    label_list.append([label_encoding[tag] for tag in sample])


print(label_list)
# print(short_dataset["ner_tags"])

[[0, 0, 0, 0, 2, 3, 3, 3, 3, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 2, 3, 3, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0], [1, 0, 2, 3, 3, 0], [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 3, 3, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [45]:
val_label_list = []

for sample in val_dataset["ner_tags"]:
    val_label_list.append([label_encoding[tag] for tag in sample])

test_label_list = []

for sample in test_dataset["ner_tags"]:
    test_label_list.append([label_encoding[tag] for tag in sample])

In [48]:
def tokenize_and_align_labels(short_dataset, list_name):
    tokenized_inputs = tokenizer(short_dataset["tokens"], truncation=True, is_split_into_words=True) ## For some models, you may need to set max_length to approximately 500.

    labels = []
    for i, label in enumerate(list_name):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [18]:
tokenized_datasets = tokenize_and_align_labels(short_dataset, label_list)
print(tokenized_datasets)
# print(i[0] for i in tokenized_datasets["token_type_ids"])

{'input_ids': [[101, 2005, 2023, 3800, 1996, 22836, 2402, 5381, 23011, 4094, 1006, 1043, 18863, 2015, 1007, 2001, 2764, 1012, 102], [101, 1996, 2206, 19389, 12955, 2020, 7594, 1024, 2358, 9626, 9080, 6204, 6651, 1006, 28177, 1010, 9587, 2140, 1044, 2475, 2080, 1049, 1011, 1016, 1055, 1011, 1015, 1007, 1010, 9099, 16781, 3446, 1006, 1041, 1010, 3461, 4747, 1044, 2475, 2080, 1049, 1011, 1016, 1055, 1011, 1015, 1007, 1010, 5658, 7760, 6038, 10760, 4588, 3446, 1006, 1052, 2078, 1010, 1166, 5302, 2140, 1049, 1011, 1016, 1055, 1011, 1015, 1007, 1998, 6970, 16882, 2522, 2475, 6693, 2522, 2475, 1006, 25022, 1010, 1166, 5302, 2140, 1049, 1011, 1016, 1055, 1011, 1015, 1007, 1012, 102], [101, 3576, 1044, 28873, 2035, 10448, 7382, 9816, 10960, 12192, 5258, 1999, 1996, 4292, 1997, 2529, 3393, 6968, 10085, 17250, 28873, 1006, 1044, 2721, 1007, 1516, 10349, 2035, 23924, 7416, 2278, 5024, 5812, 1998, 7872, 3526, 22291, 3370, 1006, 8040, 2102, 1007, 1031, 1017, 1010, 1018, 1033, 1012, 102], [101, 4958,

In [49]:
tokenized_val_datasets = tokenize_and_align_labels(val_dataset, val_label_list)
tokenized_test_datasets = tokenize_and_align_labels(test_dataset, test_label_list)

In [19]:
epochs = 6
batch_size = 4
learning_rate = 2e-5

In [81]:
%pip install transformers[torch]

   ---------------------------------------- 0.0/290.1 kB ? eta -:--:--
   --------- ------------------------------ 71.7/290.1 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 290.1/290.1 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback

In [30]:
model_name = "bert-base-uncased"
args = TrainingArguments(
    f"BERT-finetuned-NER",
    # evaluation_strategy = "epoch", ## Instead of focusing on loss and accuracy, we will focus on the F1 score
    evaluation_strategy ='steps',
    eval_steps = 7000,
    save_total_limit = 3,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.001,
    save_steps=35000,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True
)

In [40]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [24]:
%pip install seqeval

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.6 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 529.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16185 sha256=842d7b88b1cf707a7bec1c8e29c16d7903d9d62d1d3dd4d18e3077ee51934570
  Stored in directory: c:\users\gk00554\appdata\local\pip\cache\wheels\1a\67\4a\ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


In [25]:
metric = load_metric("seqeval")

c:\Users\gk00554\.conda\envs\nlp24\lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [26]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [54]:
def turn_dict_to_list_of_dict(d):
    new_list = []

    for labels, inputs in zip(d["labels"], d["input_ids"]):
        entry = {"input_ids": inputs, "labels": labels}
        new_list.append(entry)

    return new_list

In [36]:
# print(tokenized_datasets["labels"])

# new = []

# for labels, inputs in zip(tokenized_datasets["labels"], tokenized_datasets["input_ids"]):
#     entry = {"input_ids": inputs, "labels": labels}
#     new.append(entry)

[[-100, 0, 0, 0, 0, 2, 3, 3, 3, 3, 0, 1, 1, 1, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 3, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 1, 1, 0, 2, 3, 3, 3, 0, -100], [-100, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 1, 1, 1, 0, 0, -100], [-100, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

In [55]:
tokenised_train = turn_dict_to_list_of_dict(tokenized_datasets)
tokenised_val = turn_dict_to_list_of_dict(tokenized_val_datasets)
tokenised_test = turn_dict_to_list_of_dict(tokenized_test_datasets)

In [37]:
print(tokenised_train)

[{'input_ids': [101, 2005, 2023, 3800, 1996, 22836, 2402, 5381, 23011, 4094, 1006, 1043, 18863, 2015, 1007, 2001, 2764, 1012, 102], 'labels': [-100, 0, 0, 0, 0, 2, 3, 3, 3, 3, 0, 1, 1, 1, 0, 0, 0, 0, -100]}, {'input_ids': [101, 1996, 2206, 19389, 12955, 2020, 7594, 1024, 2358, 9626, 9080, 6204, 6651, 1006, 28177, 1010, 9587, 2140, 1044, 2475, 2080, 1049, 1011, 1016, 1055, 1011, 1015, 1007, 1010, 9099, 16781, 3446, 1006, 1041, 1010, 3461, 4747, 1044, 2475, 2080, 1049, 1011, 1016, 1055, 1011, 1015, 1007, 1010, 5658, 7760, 6038, 10760, 4588, 3446, 1006, 1052, 2078, 1010, 1166, 5302, 2140, 1049, 1011, 1016, 1055, 1011, 1015, 1007, 1998, 6970, 16882, 2522, 2475, 6693, 2522, 2475, 1006, 25022, 1010, 1166, 5302, 2140, 1049, 1011, 1016, 1055, 1011, 1015, 1007, 1012, 102], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2

In [56]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenised_train,
    eval_dataset=tokenised_val,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [57]:
trainer.train()

100%|██████████| 150/150 [05:08<00:00,  2.05s/it]

{'train_runtime': 308.1618, 'train_samples_per_second': 1.947, 'train_steps_per_second': 0.487, 'train_loss': 0.06668521245320638, 'epoch': 6.0}


TrainOutput(global_step=150, training_loss=0.06668521245320638, metrics={'train_runtime': 308.1618, 'train_samples_per_second': 1.947, 'train_steps_per_second': 0.487, 'train_loss': 0.06668521245320638, 'epoch': 6.0})

In [58]:
predictions, labels, _ = trainer.predict(tokenised_test)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

 97%|█████████▋| 38/39 [00:15<00:00,  2.56it/s]c:\Users\gk00554\.conda\envs\nlp24\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: [0, 0, 0, 0, 2, 3, 3, 3, 3, 0, 1, 0, 0, 0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\gk00554\.conda\envs\nlp24\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: [0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 2, 3, 3, 1, 0, 1, 0, 0, 0, 0, 0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\gk00554\.conda\envs\nlp24\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\gk00554\.conda\envs\nlp24\lib\site-packages\seqeval\metrics\sequence_label

{'0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 2, 3, 3, 1, 0, 1, 0, 0, 0, 0, 0, 0]': {'precision': 0.5760869565217391,
  'recall': 0.7940074906367042,
  'f1': 0.6677165354330709,
  'number': 267},
 '0, 0, 0, 0, 2, 3, 3, 3, 3, 0, 1, 0, 0, 0, 0]': {'precision': 0.5187861271676301,
  'recall': 0.6697761194029851,
  'f1': 0.5846905537459284,
  'number': 536},
 '0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0]': {'precision': 0.4789473684210526,
  'recall': 0.610738255033557,
  'f1': 0.5368731563421829,
  'number': 149},
 '1, 0, 2, 3, 3, 0]': {'precision': 0.4523809523809524,
  'recall': 0.5891472868217055,
  'f1': 0.5117845117845118,
  'number': 129},
 'overall_precision': 0.5204513399153737,
 'overall_recall': 0.6827012025901943,
 'overall_f1': 0.5906362545018007,
 'overall_accuracy': 0.9043752819124944}